In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder
from imblearn.pipeline import Pipeline
from sklearn.base import clone
from imblearn.over_sampling import SMOTE
import numpy as np
import joblib
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
df = pd.read_csv("Modeling_4-23.csv")

In [6]:
ohe = OneHotEncoder()
df = pd.get_dummies(df, columns=['Degree'], dtype=int)

In [8]:
X = df.loc[:, df.columns!='Depression']
y = df['Depression']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27845 entries, 0 to 27844
Data columns (total 57 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Age                                        27845 non-null  int64  
 1   Academic Pressure                          27845 non-null  int64  
 2   CGPA                                       27845 non-null  float64
 3   Study Satisfaction                         27845 non-null  int64  
 4   Work/Study Hours                           27845 non-null  int64  
 5   Financial Stress                           27845 non-null  int64  
 6   Sleep Duration_encoded                     27845 non-null  int64  
 7   Dietary Habits_encoded                     27845 non-null  int64  
 8   Depression                                 27845 non-null  int64  
 9   Have you ever had suicidal thoughts ?_No   27845 non-null  int64  
 10  Have you ever had suic

In [12]:
X_train_LR , X_test_LR , y_train_LR , y_test_LR = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
LR = LogisticRegression()

In [16]:
LR.fit(X_train_LR, y_train_LR)
y_prediction_LR = LR.predict(X_test_LR)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
print(f"Logistic Regression\n{classification_report(y_test_LR, y_prediction_LR)}")

Logistic Regression
              precision    recall  f1-score   support

           0       0.84      0.77      0.80      2316
           1       0.85      0.89      0.87      3253

    accuracy                           0.84      5569
   macro avg       0.84      0.83      0.84      5569
weighted avg       0.84      0.84      0.84      5569



## GRID SEARCH

In [23]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_LR_cv = cross_val_score(LR, X_train_LR, y_train_LR, cv=cv, scoring='recall')
print(f"Fold Metrics:\n{best_LR_cv}\n")
print(f"Average Score:\n{best_LR_cv.mean()}\n")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Fold Metrics:
[0.89340491 0.88305215 0.89378834 0.87921779 0.88611963]

Average Score:
0.8871165644171779



/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

In [ ]:
from sklearn.metrics import recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import numpy as np

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_recall = 0
best_confusion_matrix = None
fold_recalls = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model = clone(best_RF)
    model.fit(X_tr, y_tr)
    
    y_pred = model.predict(X_val)
    recall = recall_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)
    
    fold_recalls.append(recall)
    
    print(f"Fold {fold} Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{cm}\n")
    
    if recall > best_recall:
        best_recall = recall
        best_confusion_matrix = cm

print(f"All Fold Recalls: {fold_recalls}")
print(f"Mean Recall: {np.mean(fold_recalls):.4f}")
print(f"Best Fold Recall: {best_recall:.4f}")
print(f"Best Confusion Matrix:\n{best_confusion_matrix}")